In [1]:
import os
import numpy as np
import pandas as pd
from mediapipe import solutions

from video_converter import Video2DataFrame
from custom_pose_landmarks import CustomPoseLandmark

In [ ]:
# # Prepare paths
# data_path = '../data/'
# video_path = os.path.join(data_path, 'raw/squat')

# # Read personal data from excel
# personal_data = pd.read_excel(os.path.join(data_path, 'PersonalData.xlsx'))

# # Processed data path
# processed_data = os.path.join(data_path, 'processed')

In [2]:
# Selected values of pose landmarks corresponding to PoseLandmark class from MediaPipe library
values = [0, 11, 12, 13, 14, 15, 16, 19, 20, 23, 24, 25, 26, 27, 28, 31, 32]

# Custom pose landmark names and their connections
landmarks = {
    'THORAX': ['NOSE'],
    'PELVIS': ['LEFT_HIP', 'RIGHT_HIP'],}

# MediaPipe solutions
mp_drawing = solutions.drawing_utils
mp_pose = solutions.pose

custom_pose = CustomPoseLandmark(mp_pose, values, landmarks)
conv = Video2DataFrame(mp_pose, mp_drawing, custom_pose)

In [23]:
class Graph():
    """
    
    """
    def __init__(self, layout, strategy='uniform', max_distance=1, dilation=1):
        # Init
        self.max_distance = max_distance
        self.dilation = dilation
        
        self.graph_edges, self.graph_center = self.get_edges(layout)
        self.adjacency = self.get_adjacency_matrix()


    def get_edges(self, layout):
        """
        
        """
        # Extract the number of nodes in the layout
        self.num_node = layout.num_elements()
        # Create a self-loop for each node
        self_link = [(i, i) for i in range(self.num_node)]
        # Extract connections from custome layout
        neighbor_link = list(layout.get_connections())

        # Prepare the edges and center of the graph
        edges = self_link + neighbor_link
        center = 1

        return edges, center


    def get_adjacency_matrix(self):
        """ 
        
        """
        # Prepare adjacency matrix storage based on the number of nodes in the graph
        adjacency_matrix = np.zeros((self.num_node, self.num_node))

        # Fill the matrix based on the list of edges
        for i, j in self.graph_edges:
            adjacency_matrix[i, j] = 1
            adjacency_matrix[j, i] = 1

        return adjacency_matrix

    
    def get_distance_matrix(self):
        """ 
        
        """
        # Prepare a hop distance matrix storage
        distance_matrix = np.zeros_like(self.adjacency) + np.inf

        # Calculate the consecutive powers of the adjacency matrix
        transfer_matrix = [
            np.linalg.matrix_power(self.adjacency, d) for d in range(self.max_distance + 1)
        ]

        arrive_matrix = (np.stack(transfer_matrix) > 0)

        # Calculate hop distances for each node
        for d in range(self.max_distance, -1, -1):
            distance_matrix[arrive_matrix[d]] = d

        return distance_matrix


    def normalize(self, matrix):
        """ 
        normalize undirected graph, symmetry matrix
        """
        # Calculate degree (number of links) for each node
        degree_vector = np.sum(matrix, 0)

        # Prepare a diagonal matrix storage
        diagonal = np.zeros((self.num_node, self.num_node))
        
        # Calculate diagonal matrix
        for i in range(self.num_node):
            if degree_vector[i] > 0:
                diagonal[i, i] = degree_vector[i] ** (-0.5)
    
        normalized_matrix = np.dot(np.dot(diagonal, matrix), diagonal)

        return normalized_matrix
    

    def label_mapping(self, strategy):
        """ 
        
        """
        # 
        valid_hop = range(0, self.max_distance + 1, self.dilation)

        # 
        adjacency = self.get_adjacency_matrix()
        distance = self.get_distance_matrix()

        # Extend adjacency matrix based on hop distances
        for hop in valid_hop:
            adjacency[distance == hop] = 1

        # Normalize adjacency matrix before implementing the strategy
        normalized = self.normalize(adjacency)

        if strategy == 'uniform':
            labels = np.expand_dims(normalized, axis=0)
        
        elif strategy == 'distance':
            labels = np.zeros((len(valid_hop), self.num_node, self.num_node))

            for i, hop in enumerate(valid_hop):
                labels[i][distance == hop] = normalized[distance == hop]

        elif strategy == 'spatial':
            pass

        else:
            raise ValueError('strategy error')
        
        return labels

In [25]:
# Selected values of pose landmarks corresponding to PoseLandmark class from MediaPipe library
values = [0, 11, 12, 13]

# Custom pose landmark names and their connections
landmarks = {
    'THORAX': ['NOSE']}

# MediaPipe solutions
mp_drawing = solutions.drawing_utils
mp_pose = solutions.pose

custom_pose = CustomPoseLandmark(mp_pose, values, landmarks)
# conv = Video2DataFrame(mp_pose, mp_drawing, custom_pose)

In [26]:
custom_pose.get_connections()

{(1, 2), (1, 3), (4, 0)}

In [39]:
graph = Graph(
    layout=custom_pose,
    strategy='distance',
    max_distance=1,
    dilation=1
)

In [40]:
graph.adjacency

array([[1., 0., 0., 0., 1.],
       [0., 1., 1., 1., 0.],
       [0., 1., 1., 0., 0.],
       [0., 1., 0., 1., 0.],
       [1., 0., 0., 0., 1.]])

In [41]:
graph.get_adjacency_matrix()

array([[1., 0., 0., 0., 1.],
       [0., 1., 1., 1., 0.],
       [0., 1., 1., 0., 0.],
       [0., 1., 0., 1., 0.],
       [1., 0., 0., 0., 1.]])

In [42]:
graph.get_distance_matrix()

array([[ 0., inf, inf, inf,  1.],
       [inf,  0.,  1.,  1., inf],
       [inf,  1.,  0., inf, inf],
       [inf,  1., inf,  0., inf],
       [ 1., inf, inf, inf,  0.]])

In [44]:
graph.label_mapping(strategy='uniform')

array([[[0.5       , 0.        , 0.        , 0.        , 0.5       ],
        [0.        , 0.33333333, 0.40824829, 0.40824829, 0.        ],
        [0.        , 0.40824829, 0.5       , 0.        , 0.        ],
        [0.        , 0.40824829, 0.        , 0.5       , 0.        ],
        [0.5       , 0.        , 0.        , 0.        , 0.5       ]]])

In [45]:
graph.label_mapping(strategy='distance')

array([[[0.5       , 0.        , 0.        , 0.        , 0.        ],
        [0.        , 0.33333333, 0.        , 0.        , 0.        ],
        [0.        , 0.        , 0.5       , 0.        , 0.        ],
        [0.        , 0.        , 0.        , 0.5       , 0.        ],
        [0.        , 0.        , 0.        , 0.        , 0.5       ]],

       [[0.        , 0.        , 0.        , 0.        , 0.5       ],
        [0.        , 0.        , 0.40824829, 0.40824829, 0.        ],
        [0.        , 0.40824829, 0.        , 0.        , 0.        ],
        [0.        , 0.40824829, 0.        , 0.        , 0.        ],
        [0.5       , 0.        , 0.        , 0.        , 0.        ]]])